cnn 是三維的
要看srgan，怎麼規劃input

In [39]:
#training core
from torch import nn
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tqdm
import torch
import numpy as np
from torch.utils.data import Dataset,DataLoader


In [40]:
class cnn_fnn_rssi(nn.Module):
    def __init__(self):
        super(cnn_fnn_rssi,self).__init__()
        self.fnn = nn.Sequential(
            nn.Conv2d(1,1,(2,1),stride=1,padding=0),
            #nn.LayerNorm(51),
            nn.BatchNorm2d(),
            nn.ReLU()
        )
        
        self.apply(self._init_weights)
    
    #weight initialize
    def _init_weights(self,module):
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)
            module.bias.data.zero_()

    def forward(self, x):
        output = self.fnn(x)
        return output


In [41]:
class csi_rssi_cnn_dataset(Dataset):
    def __init__(self, data: np.array):
        self.uavdata = torch.from_numpy(data[0]).float()
        self.iotdata = torch.from_numpy(data[1,:,1:]).float()
        self.length = len(self.uavdata)-1
    def __getitem__(self, index):
        data, label = self.uavdata[index:index+2].unsqueeze(0), self.iotdata[index]
        return data, label
    def __len__(self):
        return self.length

In [42]:
trainingset = np.load("normalized_training_set.npy")
ds= csi_rssi_cnn_dataset(trainingset)
data_loader = DataLoader(dataset = ds,batch_size=1,shuffle= True)
checkpoint =100
epoch = 500
Model = cnn_fnn_rssi()
Model.cuda()
Model.float()
optimizer = optim.Adam(Model.parameters(), lr=1e-3, betas=(0.9, 0.999), eps=1e-8)
MSE_loss = nn.MSELoss(reduction="mean")
MSE_loss.cuda()
Model.train()
n_iters = []
losses = []
count = 0
for n_iter in tqdm(range(epoch), desc = "training"):
    train_loss = 0.0
    
    for batch_idx, (data, target) in enumerate(data_loader):
        data, target = Variable(data), Variable(target)
        data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = Model(data)
        loss = MSE_loss(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
     
    n_iters.append(n_iter)
    losses.append(train_loss)
    count += 1
    if count % checkpoint == 0:
        file_name = "model"+str(count)+".pth"
        torch.save(Model,file_name)
    torch.save(Model,"model_final.pth")

np.save("losses",np.array([n_iters,losses]))

TypeError: _BatchNorm.__init__() missing 1 required positional argument: 'num_features'